### Imports

In [ ]:
# PROCESSING DATA IMPORTS
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta
import scipy.constants

# SYSTEM 
import os
import sys
from pathlib import Path
import time

# VISUALISATION
import matplotlib.pyplot as plt
%matplotlib inline

# IMPORT UTILITY FUNCIONS
sys.path.append('./src/') # append path to ./src/ for following imports
from dl_utils import *
from df_utils import *
from misc_utils import *
import kepler_utils as kutls
import ana_utils as autls

# ML IMPORTS
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

import statsmodels.tsa.stattools as sts
import statsmodels.graphics.tsaplots as sgt

import pmdarima as pm

from scipy.optimize import curve_fit

# SPECIFY WORKING DIRECTORY (LOCATION OF SATELLITE .csv FILES)
working_dir = os.getcwd()

# CONSTANTS 
M_earth = 5.972E24 # earth mass [kg]
G = scipy.constants.G # grav constant [m^3 s^{-2} kg^{-1}]
mu = G*M_earth # standard grav parameter for m (mass moving object) << M_earth [m^3 s^{-2}]  

print('imports complete')

In [ ]:

def drop_dupl_idx(df):
    return  (df
             .reset_index()
             .drop_duplicates(subset='time_stamp',keep='last')
             .set_index('time_stamp'))

def load(path_to_csv:str):
    """
    Loads and returns the DORIS .csv-file in single DataFrame (with DateTime index) with sorted index
    """
    dtypes = {'ctr_id':'str','sat_id':'str','time_stamp':'str','x':'float','y':'float','z':'float','vx':'float','vy':'float','vz':'float'}
    df = pd.read_csv(path_to_csv,usecols=range(1,10),index_col=[2],dtype=dtypes,parse_dates=['time_stamp'])
    return df.sort_index() # sort datetime index

def preprocess(df,name:str):
    """
    """
    sat = df[df.columns[2:]].copy()
    
    # CONVERT POS TO [km]->[m] (MULTIPLY BY 1000) AND VELOCITY TO [dm/s]->[m/s] (DIVIDE BY 10)
    sat[['x','y','z']] = sat[['x','y','z']] * 1_000
    sat[['vx','vy','vz']] = sat[['vx','vy','vz']] / 10

    # DROP DUPLICATE INDICES
    sat = drop_dupl_idx(sat) 

    # RESAMPLE 
    if len(sat.index.diff()[1:].value_counts()) > 1:
        print(f'{name} has observational gaps')
    #     sat_og = sat.copy()
    #     sat = sat.resample('1min').mean() 
    #     for col in sat.columns:
    #         sat[col] = fft_resample(sat_og[col])

    # CHECK FOR NULL VALUES
    if sat.isnull().sum().sum() != 0:
        print('ATTENTION: Null values encountered')

    return sat

def orbital_elements(sat,modus:str):
    """
    Computes osculating elements (i, Omega, omega, a, e, h) from the satellite's position and velocity
    """

    sat_cp = sat.copy()

    idx = sat.index

    if modus == 'classical':
    
        i = pd.DataFrame(kutls.i(sat),columns=['i']).set_index(idx)

        Omega = pd.DataFrame(kutls.Omega(sat),columns=['Omega']).set_index(idx)

        omega = pd.DataFrame(kutls.omega(sat),columns=['omega']).set_index(idx)

        a = pd.DataFrame(kutls.a(sat),columns=['a']).set_index(idx)

        e = pd.DataFrame(kutls.e(sat),columns=['e']).set_index(idx)

        nu = pd.DataFrame(kutls.nu(sat),columns=['nu']).set_index(idx)

        return pd.concat([sat_cp,a,e,nu,i,Omega,omega],axis=1)
    
    if modus=='mean':

        E = pd.DataFrame(kutls.E(sat),columns=['E']).set_index(idx)

        M = pd.DataFrame(kutls.M(sat),columns=['M']).set_index(idx)

        n = pd.DataFrame(kutls.n(sat),columns=['n']).set_index(idx)

        return pd.concat([sat_cp,E,M,n],axis=1)

    if modus == 'modified':

        p = pd.DataFrame(kutls.p(sat),columns=['p']).set_index(idx)

        f = pd.DataFrame(kutls.f(sat),columns=['f']).set_index(idx)

        g = pd.DataFrame(kutls.g(sat),columns=['g']).set_index(idx)

        q1 = pd.DataFrame(kutls.q1(sat),columns=['q1']).set_index(idx)

        q2 = pd.DataFrame(kutls.q2(sat),columns=['q2']).set_index(idx)

        L = pd.DataFrame(kutls.L(sat),columns=['L']).set_index(idx)

        return pd.concat([sat_cp,p,f,g,q1,q2,L],axis=1)


### Loading and Preprocessing Data

In [ ]:
# LOAD DATA 
s6ssa = load(working_dir+'/sat/s6ssa2024/s6assa_20_24.csv')

print('\n loading data complete')


### Preprocess Data

In [ ]:
s6assa = preprocess(s6ssa,'s6assa')

In [ ]:
# MANEUVERS
maneuvers = pd.read_csv('/Users/donny/Desktop/TTM/training/IDS/references/maneuvers_schedule.csv',index_col=0)
maneuvers.start = pd.to_datetime(maneuvers.start)
maneuvers.end = pd.to_datetime(maneuvers.end)

manja32024 = maneuvers[(maneuvers.sat_id == 'ja3') & (maneuvers.start.dt.year.isin(range(2020,2025)))].drop(['sat_id','end'],axis=1).set_index('start').sort_index()
mans6a2024 = maneuvers[(maneuvers.sat_id == 's6a') & (maneuvers.start.dt.year.isin(range(2020,2025)))].drop(['sat_id','end'],axis=1).set_index('start').sort_index()
manh2c2024 = maneuvers[(maneuvers.sat_id == 'h2c') & (maneuvers.start.dt.year.isin(range(2020,2025)))].drop(['sat_id','end'],axis=1).set_index('start').sort_index()
manh2d2024 = maneuvers[(maneuvers.sat_id == 'h2d') & (maneuvers.start.dt.year.isin(range(2020,2025)))].drop(['sat_id','end'],axis=1).set_index('start').sort_index()

print('loading maneuvers complete')

loading maneuvers complete
